In [1]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm

import plotly.graph_objs as go
from plotly.subplots import make_subplots
import plotly.io as pio

from xgboost import XGBRegressor
import pickle
from tensorflow.keras.models import load_model

import warnings
warnings.filterwarnings("ignore")

In [2]:
def noised(X, σ):
    noise = np.random.normal(0, σ, size=(X.shape[0], X.shape[1]))
    return X + noise

In [3]:
# models = []
# models.append(pickle.load(open("models/xgb_energy_ideal_opt_16.dat", "rb")))
# models.append(pickle.load(open("models/xgb_energy_real_opt_16.dat", "rb")))
# models.append(load_model("models/5M_ideal.h5"))
# models.append(load_model("models/5M_real.h5"))

# scalers = []
# scalers.append(pickle.load(open('models/scaler_ideal.pkl','rb')))
# scalers.append(pickle.load(open('models/scaler_real.pkl','rb')))

In [4]:
model = load_model("models/5M_real.h5")
scaler = pickle.load(open('models/scaler_real.pkl','rb'))

In [5]:
# opt_features = ['AccumCharge', 'R_cht', 'z_cc', 'pe_std',
#                 'nPMTs', 'ht_kurtosis', 'ht_25-20p', 'R_cc',
#                 'ht_5-2p', 'pe_mean', 'jacob_cht', 'phi_cc',
#                 'ht_35-30p', 'ht_20-15p', 'pe_35p', 'ht_30-25p']

# opt_features += ['edep']

opt_features = ['AccumCharge', 'nPMTs', 'R_cc', 'R_cht', 'pe_mean',
                'pe_std', 'pe_skew', 'pe_kurtosis', 'pho_cc', 'pho_cht',
                'ht_2p', 'ht_5p', 'ht_10p', 'ht_15p', 'ht_20p',
                'ht_25p', 'ht_30p', 'ht_35p', 'ht_40p', 'ht_45p',
                'ht_50p', 'ht_55p', 'ht_60p', 'ht_65p', 'ht_70p',
                'ht_75p', 'ht_80p', 'ht_85p', 'ht_90p', 'ht_95p',
                'edep']

In [6]:
%%time
path='/mnt/cephfs/ml_data/mc_2021/'
sigmas = np.linspace(0, 0.03, 31)
energies = [0, 0.1, 0.3, 0.6] + list(range(1, 11))
y_true_array = []
y_pred_array = []
for j in tqdm(range(len(sigmas)), "Sigmas..."):
    y_true = []
    y_pred = []
    for energy in tqdm(energies, "Energies...", leave=False):
        test = pd.read_csv(f'{path}processed_data/ProcessedTestReal/{energy}MeV.csv.gz')
        test = test[test['edepR'] < 17.2]
        edep = np.array(test['edep'])
        X_test = test[opt_features].iloc[:, :-1]
        X_test = scaler.transform(X_test)
        X_test = noised(X_test, sigmas[j])
        edep_preds = model.predict(X_test).flatten()

        y_true.append(edep)
        y_pred.append(edep_preds)
    y_true_array.append(y_true)
    y_pred_array.append(y_pred)

Sigmas...:   0%|          | 0/31 [00:00<?, ?it/s]

Energies...:   0%|          | 0/14 [00:00<?, ?it/s]

Energies...:   0%|          | 0/14 [00:00<?, ?it/s]

Energies...:   0%|          | 0/14 [00:00<?, ?it/s]

Energies...:   0%|          | 0/14 [00:00<?, ?it/s]

...

Energies...:   0%|          | 0/14 [00:00<?, ?it/s]

Energies...:   0%|          | 0/14 [00:00<?, ?it/s]

Energies...:   0%|          | 0/14 [00:00<?, ?it/s]

Energies...:   0%|          | 0/14 [00:00<?, ?it/s]

CPU times: user 2h 30min 19s, sys: 16min 49s, total: 2h 47min 8s
Wall time: 1h 58min 16s


In [7]:
# for j in tqdm(range(2), "Options..."):
#     if j == 0:
#         name = 'ProcessedTestIdeal'
#     elif j == 1:
#         name = 'ProcessedTestReal'
#     y_true = []
#     y_pred = []
#     for energy in tqdm(energies, "Energies...", leave=False):
#         test = pd.read_csv('{}processed_data/{}/{}MeV.csv.gz'.format(path, name, energy))
#         test = test[test['edepR'] < 17.2]
#         edep = np.array(test['edep'])
#         X_test = test[opt_features].iloc[:, :-1]
# #         edep_preds = models[j].predict(noised(np.array(X_test)))
#         X_test = scalers[j].transform(X_test)
#         X_test = noised(X_test)
#         edep_preds = models[j].predict(X_test).flatten()

#         y_true.append(edep)
#         y_pred.append(edep_preds)
#     y_true_array.append(y_true)
#     y_pred_array.append(y_pred)

In [8]:
diffs = np.array([
    [y_pred_array[j][i] - y_true_array[j][i] for i in range(len(y_pred_array[0]))]
    for j in range(len(y_pred_array))
])

In [9]:
energies = [0, 0.1, 0.3, 0.6] + list(range(1, 11))
energies = np.array([1.022+i for i in energies]).round(5)
energies

array([ 1.022,  1.122,  1.322,  1.622,  2.022,  3.022,  4.022,  5.022,
        6.022,  7.022,  8.022,  9.022, 10.022, 11.022])

In [10]:
# names = ['Elecsim 5M Ideal', 'Elecsim 5M Real', 'Elecsim+Noise 5M Ideal', 'Elecsim+Noise 5M Real']
names = [f'Sigma = {sigma.round(3)}' for sigma in sigmas]

In [11]:
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from scipy.stats import norm

a_array = []
errors_array = []
for k in range(diffs.shape[0]):
    a = []
    e = []
    for i in range(diffs.shape[1]):
        fig, ax = plt.subplots()
        nbins = 150
        n, bins, patches = ax.hist(diffs[k][i], nbins, density=True, facecolor = 'grey', alpha = 0.5, label='before');
        plt.close(fig)
        centers = (0.5*(bins[1:]+bins[:-1]))
        pars, cov = curve_fit(lambda x, mu, sig : norm.pdf(x, loc=mu, scale=sig), centers, n, p0=[0,1])  
        a.append(pars)
        e.append(cov)
    a_array.append(a)
    errors_array.append(e)

In [12]:
# pio.templates.default = 'plotly_white'

# from scipy import stats
# for k in range(len(diffs)):
#     fig = go.Figure()

#     for i in range(len(diffs[k])): 
#         x = np.linspace(diffs[k][i][:22500].min(), diffs[k][i][:22500].max(), 100)
#         p = stats.norm.pdf(x, a_array[k][i][0], a_array[k][i][1])

#         fig.add_trace(go.Scatter(x=x,
#                      y=p, mode='lines', name='mu={:.3f} +- {:.3f}, sigma={:.3f} +- {:.3f}'.format(
#                          a_array[k][i][0], np.sqrt(errors_array[k][i][0][0]),
#                          a_array[k][i][1], np.sqrt(errors_array[k][i][1][1])),
#                         visible = (i==0)
#                     ))

#     for i in range(len(diffs[k])): 
#         fig.add_trace(go.Histogram(x=diffs[k][i][:22500], xbins=dict(size=0.005),
#                       showlegend=False, histnorm='probability density',
#                      visible = (i==0)
#                     ))

#     buttons = []
#     for N in range(0, len(diffs[k])): 
#         buttons.append(
#             dict(
#                  args=['visible', [False]*N + [True] + [False]*(len(diffs[k])-1-N)],
#                      label='Energy =  {} MeV'.format(energies[N]),
#                  method='restyle'
#             )
#         )
        
#     fig.update_layout(

#         xaxis = dict(
#             showline=True,
#             ticks='outside',
#             mirror=True,
#             linecolor='black',
#             showgrid=True,
#             gridcolor='grey',
#             gridwidth=0.25,
#         ),

#         yaxis = dict(
#             showline=True,
#             ticks='outside',
#             mirror=True,
#             linecolor='black',
#             tick0=0,
# #             dtick=1,
#             showgrid=True,
#             gridcolor='grey',
#             gridwidth=0.25,
#             zeroline=True,
#             zerolinecolor='black',
#             zerolinewidth=0.25
#             ),
#     )


#     fig.update_layout(
#         title = '{}'.format(names[k]),
#         xaxis_title=r"$$E_{rec} - E_{true}$$",
# #         yaxis_title="y",
#         showlegend=True,
#         updatemenus=list([
#             dict(
#                 x=0.5,
#                 y=1.2,
#                 yanchor='top',
#                 buttons=buttons
#             ),
#         ]),
#         legend=dict(
#             orientation="h",
#             yanchor="bottom",
#             y=1.05,
#             xanchor="right",
#             x=1
#         )
#     )
    
#     fig.show()

In [23]:
# def plot_results(appr=False):

#     fig = make_subplots(rows=2, cols=1,
#                         shared_xaxes=True,
#                         vertical_spacing=0.01,
#                         row_width=[0.25, 0.75]
#     )

#     for k in range(diffs.shape[0]):
#         fig.add_trace(
#             go.Scatter(
#                 x=energies,
#                 y=res[k],
#                 mode='markers',
#                 marker=dict(
# #                     color=colors[k],
# #                     symbol=symbols[k]
#                 ),
#                 showlegend=True,
#                 error_y=dict(
#                     type='data',
#                     width=10,
#                     array=error_sigma[k],
#                     visible=True
#                 ),
#                 name=names[k]
#             ), row=1, col=1
#         )

#         fig.add_trace(
#             go.Scatter(
#                 x=energies,
#                 y=bias[k],
#                 mode='markers',
#                 showlegend=False,
#                 marker=dict(
# #                     color=colors[k],
# #                     symbol=symbols[k]
#                 ),
#                 error_y=dict(
#                         type='data',
#                         width=10,
#                         array=error_mu[k],
#                         visible=True
#                 ),
#                 name=names[k]
#             ), row=2, col=1
#         )

#     if appr:
#         for k in range(len(names)):
#             fig.add_trace(
#                 go.Scatter(
#                     x=x_lin,
#                     y=func(x_lin, a[k], b[k], c[k]),
#                     mode='lines',
#                     line=dict(
#                     ),
#                     opacity=0.5,
#                     showlegend=False,
#                     name=names[k],
#                     marker=dict(
# #                         color=colors[k]
#                     )
#                 ), row=1, col=1
#             )

#     xaxis = dict(
#         showline=True,
#         ticks='outside',
#         mirror=True,
#         tick0=1,
#         dtick=1,
#         linecolor='black',
#         showgrid=True,
#         gridcolor='grey',
#         gridwidth=0.25,
#     )

#     yaxis = lambda range: dict(
#         showline=True,
#         ticks='outside',
#         mirror=True,
#         linecolor='black',
#         range=range,
#         showgrid=True,
#         gridcolor='grey',
#         gridwidth=0.25,
#         zeroline=True,
#         zerolinecolor='black',
#         zerolinewidth=0.25
#     )

#     fig.update_layout(
#         xaxis2_title="Visible energy, MeV",
#         yaxis1_title="Resolution, %",
#         yaxis2_title="Bias, %",

#         xaxis1 = xaxis,
#         xaxis2 = xaxis,
#         yaxis1 = yaxis([0, 4]),
#         yaxis2 = yaxis([-0.25, 0.25]),

#         showlegend=True,
#         font=dict(
#                 family="Times New Roman",
#                 size=18,
#         ),
#         legend=dict(
#             x=0.75,
#             y=0.99,
#             title_font_family="Times New Roman",
#             font=dict(
#                 family="Times New Roman",
#                 size=18,
#                 color="black"
#             ),
#             bordercolor="Black",
#             borderwidth=2
#         )
#     )

#     fig.show()
# #     if appr:
# #         pio.write_image(fig, 'plots/appr_results_opt_{}.pdf'.format(len(opt_features)-1), width=900, height=600, scale=1)
# #         run['output/appr_results.pdf'].upload(File.as_html(fig))
# #     else:
# #         pio.write_image(fig, 'plots/results_opt_{}.pdf'.format(len(opt_features)-1), width=900, height=600, scale=1)
# #         run['output/results.pdf'].upload(File.as_html(fig))

In [14]:
colors = ['red', 'darkviolet', 'blue', 'green']
symbols = ['hexagon-open', 'star-square', 1, 2]

In [15]:
error_sigma = []
for k in range(diffs.shape[0]):
    error = [100 * np.sqrt(errors_array[k][i][1][1]) / energies[i] for i in range(len(energies))]
    error_sigma.append(error)
    
error_mu = []
for k in range(diffs.shape[0]):
    error = [100 * np.sqrt(errors_array[k][i][0][0]) / energies[i] for i in range(len(energies))]
    error_mu.append(error)

In [16]:
res = []
bias = []
for k in range(diffs.shape[0]):
    sigma = [100 * a_array[k][i][1] / energies[i] for i in range(len(energies))]
    mu = [100 * a_array[k][i][0] / energies[i] for i in range(len(energies))]
    res.append(sigma)
    bias.append(mu)
    
# plot_results()

In [17]:
def a(x, a):
    return np.sqrt((a/x**0.5)**2)


def b(x, b):
    b_list = []
    b_list.append(np.sqrt(b**2))
    return b_list*len(x)


def c(x, c):
    return np.sqrt((c/x)**2)


def func(x, a, b, c):
    return np.sqrt((a/x**0.5)**2 + b**2 + (c/x)**2) 


def approximated(x, y, yerr):
    popt, pcov = curve_fit(func, x, y, sigma=yerr, maxfev=10**8, bounds=([0, 0, 0.25], [5, 5, 5]))
    a, b, c = popt
    #perr = np.sqrt(abs(pcov.diagonal()))

    return func(x, a, b, c), popt, pcov

In [18]:
from statsmodels.stats.moment_helpers import cov2corr

y_approximated_array = []
coefs_array = []
errors_array = []
corr_matrixes = []
for i in range(diffs.shape[0]):
    y_approximated, coefs, pcov = approximated(
        energies[1:13], res[i][1:13], error_sigma[i][1:13])
    y_approximated_array.append(y_approximated)
    coefs_array.append(coefs)
    errors_array.append(np.sqrt(abs(pcov.diagonal())))
    corr_matrixes.append(cov2corr(pcov))

corr_matrixes = np.array(corr_matrixes)

In [19]:
reindex = [0, 3, 1, 4, 2, 5]
coefs_df = pd.DataFrame(
    np.hstack((coefs_array, errors_array))
)[reindex]
coefs_df.columns = ['a', r'$\Delta a$', 'b', r'$\Delta b$', 'c', r'$\Delta c$']

a = np.array(coefs_array).T[0]
b = np.array(coefs_array).T[1]
c = np.array(coefs_array).T[2]

In [20]:
x_lin = np.linspace(0.8, 11.5, 1000)
# plot_results(appr=True)

In [21]:
coefs_df = coefs_df.round(3)
coefs_df.index = names
coefs_df[r'$\tilde{a}$'] = (coefs_df['a']**2 + (1.6 * coefs_df['b'])**2 + (coefs_df['c'] / 1.6)**2)**0.5 
coefs_df[r'$\Delta \tilde{a}$'] = np.sqrt( (coefs_df['a']*coefs_df[r'$\Delta a$'])**2 + \
                                           (2.56*coefs_df['b']*coefs_df[r'$\Delta b$'])**2 + \
                                           (coefs_df['c']*coefs_df[r'$\Delta c$'] / 2.56)**2) / coefs_df[r'$\tilde{a}$']

coefs_df[r'$\Delta \tilde{a}$'] = np.sqrt(
    coefs_df[r'$\Delta \tilde{a}$']**2 + 2 * (
        1.6**2 * coefs_df['a'] * coefs_df['b'] / coefs_df[r'$\tilde{a}$']**2 *\
        corr_matrixes[:, 0, 1] * coefs_df[r'$\Delta a$'] * coefs_df[r'$\Delta b$'] +\
        
        coefs_df['a'] * coefs_df['c'] / (coefs_df[r'$\tilde{a}$']**2 * 1.6**2) *\
        corr_matrixes[:, 0, 2] * coefs_df[r'$\Delta a$'] * coefs_df[r'$\Delta c$'] +\

        coefs_df['b'] * coefs_df['c'] / coefs_df[r'$\tilde{a}$']**2 *\
        corr_matrixes[:, 1, 2] * coefs_df[r'$\Delta b$'] * coefs_df[r'$\Delta c$']
    )
)

coefs_df

,a,$\Delta a$,b,$\Delta b$,c,$\Delta c$,$\tilde{a}$,$\Delta \tilde{a}$
Sigma = 0.0,2.316,0.139,0.827,0.054,1.474,0.285,2.821952,0.026560
Sigma = 0.001,2.330,0.133,0.821,0.051,1.442,0.283,2.822534,0.026008
Sigma = 0.002,2.351,0.127,0.814,0.050,1.439,0.274,2.834135,0.024915
Sigma = 0.003,2.321,0.136,0.827,0.052,1.507,0.280,2.832848,0.025583
Sigma = 0.004,2.347,0.140,0.823,0.055,1.503,0.277,2.850403,0.027830
Sigma = 0.005,2.354,0.132,0.821,0.052,1.544,0.260,2.863230,0.026241
Sigma = 0.006,2.389,0.135,0.820,0.054,1.487,0.283,2.879653,0.026089
Sigma = 0.007,2.381,0.134,0.822,0.053,1.597,0.259,2.897441,0.026080
Sigma = 0.008,2.430,0.124,0.808,0.048,1.586,0.256,2.925544,0.025249
Sigma = 0.009,2.443,0.130,0.813,0.052,1.644,0.251,2.952302,0.026432


In [26]:
pio.templates.default = 'plotly_white'

fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=sigmas,
        y=coefs_df[r'$\tilde{a}$']+coefs_df[r'$\Delta \tilde{a}$'],
        mode='lines',
        line=dict(color='black', width=0.75),
    )
)

fig.add_trace(
    go.Scatter(
        x=sigmas,
        y=coefs_df[r'$\tilde{a}$']-coefs_df[r'$\Delta \tilde{a}$'],
        fill='tonexty',
        fillcolor='darkgreen',
        mode='lines',
        line=dict(color='black', width=0.75)
    )
)

fig.add_trace(
    go.Scatter(
        x=sigmas,
        y=coefs_df[r'$\tilde{a}$'],
        mode='lines',#'lines+markers',
        line=dict(
            dash='dash',
            color='black'
        ),
        marker=dict(
            color='black',
            size=8,
            symbol='x'
        ),
    )
)

xaxis = lambda range: dict(
    showline=True,
    ticks='outside',
    mirror=True,
    tick0=1,
    dtick=0.005,
    range=range,
    linecolor='black',
    showgrid=True,
    gridcolor='grey',
    gridwidth=0.25,
)

yaxis = lambda range: dict(
    showline=True,
    ticks='outside',
    mirror=True,
    linecolor='black',
    range=range,
    showgrid=True,
    gridcolor='grey',
    gridwidth=0.25,
    zeroline=True,
    zerolinecolor='black',
    zerolinewidth=0.25
)

fig.update_layout(
    xaxis_title=r"$\sigma$",
    yaxis_title=r'$\tilde{a}$',
    xaxis = xaxis([0, 0.03]),
    yaxis = yaxis([2.7, 4.1]),

    showlegend=False,
    height=500,
    width=950,
    font=dict(
            family="Times New Roman",
            size=16,
    ),
)

fig.show()
pio.write_image(fig, 'plots/atilde_vs_noise.pdf', height=500, width=950)

In [29]:
# coefs_df.to_csv('plots/params_vs_noise.csv', index=False)